<p style="text-align:center"><img src="https://github.com/WindIO-Bachelorthesis/Shortterm_Forecast/blob/main/images/header.PNG?raw=true"></p>

<p style="text-align:center"><img src="https://camo.githubusercontent.com/4a300a0747dc7b5c69a83cc94ff2e2b5ad1f0f937ed3d142b171c0d4a6f8e164/68747470733a2f2f7777772e67652e636f6d2f72656e657761626c65656e657267792f73697465732f64656661756c742f66696c65732f323032302d30312f6f6e73686f72652d6865726f352e6a7067"/></p>


<h1 align="center">This script transfers the preprocessed data to the CE4IoT WindIO instance.</h1>

In [1]:
import ast
import pandas as pd
import requests
import json

#### Please fill the needed infromation.

In [2]:
pre_path = "./transfer/measurement_windio_msb-0004-a_2021-10-21.csv"
json_path = "./transfer/measurement_windio_msb-0004-a_2021-10-21.json"
instance_url = "https://windio-contact.northeurope.cloudapp.azure.com:8443/"
user_name = 'caddok'
password = 'l@ledi57'
asset_name = "AS-000003-0003"
time_series_name = "measurement_windio_msb-0002-a_2021-10-21"

#### Execute this cell to transfer your data to the CE4IoT instance.

In [3]:
df = pd.read_csv(pre_path, parse_dates=["time"])
df["time"] = pd.to_datetime(df["time"], unit='s')
df = df.to_json(json_path, orient = 'records')

with open(json_path) as json_file:
    json_data = json.load(json_file)
    
def get_csrf_token(session):
    csrf = session.cookies.get('CSRFToken')
    if csrf is None:
        session.get(instance_url + "/server/sessioninfo")
        csrf = session.cookies.get('CSRFToken')
    return csrf

with requests.Session() as session:
    session.auth = (user_name, password)
    headers = {
        "X-Csrf-Token": get_csrf_token(session)
    }
    resp = session.post(instance_url + "/iot_api/v1/assets/" + asset_name + "/time_series/" + time_series_name + "/data", json=json_data[:10], headers=headers)
    print(resp)
    resp = session.get(instance_url + '/server/__quit__')

<Response [200]>
[{'acc_x ($m/s^{2}$)': 0.0422696897, 'acc_y ($m/s^{2}$)': -0.030005604, 'time': 1634809343724, 'vel_x ($m/s$)': -0.033023286, 'vel_y ($m/s$)': -0.0107052153, 'pos_x ($m$)': 0.0826465004, 'pos_y ($m$)': 0.0047304993}, {'acc_x ($m/s^{2}$)': 0.0278943996, 'acc_y ($m/s^{2}$)': -0.0240167153, 'time': 1634809343744, 'vel_x ($m/s$)': -0.0322770287, 'vel_y ($m/s$)': -0.0112193001, 'pos_x ($m$)': 0.0816677424, 'pos_y ($m$)': 0.0044842803}, {'acc_x ($m/s^{2}$)': 0.033877809, 'acc_y ($m/s^{2}$)': 0.0041261943, 'time': 1634809343763, 'vel_x ($m/s$)': -0.0316068692, 'vel_y ($m/s$)': -0.0114004956, 'pos_x ($m$)': 0.0806889195, 'pos_y ($m$)': 0.0042306207}, {'acc_x ($m/s^{2}$)': 0.0566273877, 'acc_y ($m/s^{2}$)': -0.0383838809, 'time': 1634809343783, 'vel_x ($m/s$)': -0.0306452595, 'vel_y ($m/s$)': -0.0117229437, 'pos_x ($m$)': 0.0797077947, 'pos_y ($m$)': 0.0039697687}, {'acc_x ($m/s^{2}$)': 0.0554278029, 'acc_y ($m/s^{2}$)': -0.0018582192, 'time': 1634809343802, 'vel_x ($m/s$)': -0